# CASE STUDY - Extract Tabular Data From PDF files

An Investment analytics firm “RAVE Analytics” analyses quarterly results of various companies and advises their clients based on its analysis. Mohan works as a Data Analyst for RAVE Analytics, and his work is to get the quarterly results for various companies from their respective websites/ and store it in a given format in a database for future use. He starts with the quarterly results of “Reliance India Limited.” The quarterly results of RIL are on their website but in pdf format. He needs the final data in the following format.

|CName |CCode|Year |Quarter |Revenue |EBITDA |NetProfit |EPS|
|---- |:----|:--- |:------ |:------ |:----- |:-------- |:-----|


How will Mohan proceed? 


Mohan decided to break this problem into the following three steps.
1. Getting the input pdf files.
2. Processing the input files
3. Saving the outupt files

He created the following folders to keep the different files which are self explanatory.

1. input_pdf_RIL
2. processed_pdf_RIL
3. error_pdf_RIL
4. output_csv_RIL

He proceeded with importing libraries in python as below.

In [177]:
## Import libraries
import os
import pandas as pd
import PyPDF2
import re
import camelot as cm
from datetime import datetime

In [178]:
## Setting various folder paths

INPUTFODLER=r'D:\Projects_Data\Github\DataScienceTutorial\ExtractTableFromPDF\input_pdf_RIL'
OUTPUTFOLDER=r'D:\Projects_Data\Github\DataScienceTutorial\ExtractTableFromPDF\output_csv_RIL'
ERRORFOLDER=r'D:\Projects_Data\Github\DataScienceTutorial\ExtractTableFromPDF\error_pdf_RIL'
PROCESSEDFOLDER=r'D:\Projects_Data\Github\DataScienceTutorial\ExtractTableFromPDF\processed_pdf_RIL'

## Logic of processing input pdf files

Mohan got the quartely results file from RIL website and put it in the input folder.

After brainstorming the problem, he broke down the problem into three steps for each pdf file which is shown in the following diagram. 

In [189]:
## Read the path of all pdf files into a list
def get_all_pdf_files(folderpath=""):
    input_file_list=[]
    for file in os.listdir(folderpath):
        if file.lower().endswith(".pdf"):
            file_name= os.path.join(folderpath, file)
            input_file_list.append(file_name)
    return input_file_list

In [190]:
def find_page_number_quartely_result(filepath):
    """
     returns pagenumber of the pdf file which containes  consolidated financial performance
    param:
        filepath - path of the pdf file
        
    returns pagenumber
    
    """
    pagenumber= -1
    matchstring="CONSOLIDATED FINANCIAL PERFORMANCE"
    with open(filepath, "rb") as f:
        pdfDoc = PyPDF2.PdfFileReader(f,"rb")
        for i in range(0,pdfDoc.getNumPages()):
            content=""
            content = pdfDoc.getPage(i).extractText()
            #print(content)
            content=content.replace("\n","")
            ResSearch = re.search(matchstring, content)
            if ResSearch is not None:
                pagenumber=i+1
                break
    return pagenumber

In [191]:
def extract_quarterly_result_table_pdf(page_number,filepath):
    """
    extract raw consolidated financial performance table from the input pdf file
    
    param:
        page_number - page number where consolidated financial performance table appears
        filepath - path of the pdf file
        
    returns df
    
    """
    df=pd.DataFrame()
    ### camelot library is used
    var= cm.read_pdf(filepath,pages=str(page_number),copy_text=["v"])
    if(len(var)==1):
        df=var[0].df
    return df

In [193]:
def transform_data(inputdf):
    """
    This function takes the cleaned dataframe extracted from pdf and returns a dictionary with desired data
    
    param:
        inputdf - dataframe
        
        
    returns outputdict
    
    """
    cname= "Reliance Industries Limited"
    ccode="RIL"
    x=inputdf.columns[0].split("_")[1]
    fiscal_year= '20'+ re.sub(r'[FY ]','',x)
    quarter=inputdf.columns[0].split("_")[0]
    quarter_no= re.sub(r'[ Q]','',quarter)
    revenue=inputdf.loc['Revenue'].values[0]
    ebitda= inputdf.loc['EBITDA'].values[0]
    net_profit= inputdf.loc['Net_Profit'].values[0]
    eps= inputdf.loc['EPS'].values[0].replace('#','')
    outputdict= {'CName':cname,
                 'CCode':ccode,
                 'Fiscal_Year':fiscal_year,
                 'Quarter_No':quarter_no,
                 'Revenue':revenue,
                 'EBITDA':ebitda,
                 'Net_Profit':ebitda,
                 'EPS':eps
                }
    return outputdict

In [194]:
def clean_dataframe(inputdf):
    """
    clean the complete tabular data taken out from pdf and returns dataframe with only desired data
    
    
    returns outputdf
    """
    outputdf= pd.DataFrame()
    col_list=inputdf.loc[0].tolist()
    col_list= [item.replace('\n','_').replace(' ','') for item in col_list]
    inputdf.columns=col_list
    inputdf.drop([0],inplace=True)
    inputdf['KPI']= ['Revenue','EBITDA','Net_Profit','EPS']
    inputdf.set_index('KPI',inplace=True)
    inputdf.drop(df.columns[0],axis=1,inplace=True)
    outputdf = pd.DataFrame(index=inputdf.index,columns=[inputdf.columns[0]],data=df[inputdf.columns[0]])
    return outputdf

In [187]:
ril_dict_list=[]
pdf_file_path_list=get_all_pdf_files(INPUTFODLER)
for filepath in pdf_file_path_list:
    try:
        page_number= find_page_number_quartely_result(filepath)
        df= extract_quarterly_result_table_pdf(page_number,filepath)
        newdf= clean_dataframe(df)
        ril_dict= transform_data(newdf)
        ril_dict_list.append(ril_dict)
        ## move TO PROCESSED FOLDER
        filename= os.path.basename(filepath)
        os.rename(filepath, os.path.join(PROCESSEDFOLDER,filename))
    except:
        ### MOVE TO ERROR FOLDER
        filename= os.path.basename(filepath)
        os.rename(filepath, os.path.join(ERRORFOLDER,filename))

if ril_dict_list:
    today = datetime.now()
    datestr=today.strftime("%b%d%Y%H%M%S")
    csv_name= "RIL" +"_" +datestr +".csv"
    csv_file_name= os.path.join(OUTPUTFOLDER, csv_name)
    ril_df= pd.DataFrame(ril_dict_list)
    ril_df.to_csv(csv_file_name,index=True)